In [ ]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import os
from datetime import datetime, timedelta
import logging
import time
import json

APIKEY = "" # Replace with your actual API key

In [236]:
# Read CSV file
df = pd.read_csv('station66_distinct_clean.csv')
df2 = pd.read_csv('thailand_province_amphoe_tambon.csv')
    
# Clean THAI NUMBER in location
thai_numbers = str.maketrans("๐๑๒๓๔๕๖๗๘๙", "0123456789")
df["location"] = df["location"].str.translate(thai_numbers)
# Clean texts in location
keywords = ["เต็นท์", "เต้นท์", "ปะรำ", "บริเวณ"]
for text in keywords:
    df["location"] = df["location"].str.replace(text, "", regex=False).str.strip()
# Clean bracketed numbers in location
df["location"] = df["location"].str.replace(r"\(\d+\)", " ", regex=True).str.strip()
# Clean bracketed single characters in location
df["location"] = df["location"].str.replace(r"\([ก-ฮ]\)", " ", regex=True).str.strip()
# Clean double spaces in location
df["location"] = df["location"].str.replace(r"\s+", " ", regex=True).str.strip()
# Clean space before/after parenthese in location
df["location"] = df["location"].str.replace(r"\s+\(", "(", regex=True).str.replace(r"\)\s+", ")", regex=True).str.strip()
# Add space before/after parenthese in location
df["location"] = df["location"].str.replace(r"\(", " (", regex=True).str.replace(r"\)", ") ", regex=True).str.strip()
# CLean space before/after hyphen in location
df["location"] = df["location"].str.replace(r"\s*-\s*", "-", regex=True).str.strip()
# Clean spaces after text
df["location"] = df["location"].str.replace(r"\s+", " ", regex=True).str.strip()

# Save cleaned df to new CSV
df.to_csv('station66_distinct_clean_super.csv', index=False)

# Clean df.registrar text in all rows
keywords = ["ท้องถิ่น", "เทศบาล", "อำเภอ", "ตำบล", "เขต", "แขวง"]
df["proposed_district"] = df["registrar"]
for text in keywords:
    df["proposed_district"] = df["proposed_district"].str.replace(text, "", regex=False).str.strip()

display(df, df2.head())

,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,proposed_district
0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช,13.752802,100.489581,พระนคร
1,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์,13.755687,100.490966,พระนคร
2,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,13.745237,100.495748,พระนคร
3,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดพระเชตุพน ถนนมหาราช,13.745582,100.491293,พระนคร
4,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100102,วังบูรพาภิรมย์,1,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,13.743462,100.498851,พระนคร
...,...,...,...,...,...,...,...,...,...,...,...
81422,96,นราธิวาส,9699,ท้องถิ่นเทศบาลเมืองนราธิวาส,960101,บางนาค,1,หอประชุมวิทยาลัยเทคนิคนราธิวาส,NaN,NaN,เมืองนราธิวาส
81423,96,นราธิวาส,9699,ท้องถิ่นเทศบาลเมืองนราธิวาส,960101,บางนาค,1,อาคารอเนกประสงค์โรงเรียนเทศบาล 5,NaN,NaN,เมืองนราธิวาส
81424,96,นราธิวาส,9699,ท้องถิ่นเทศบาลเมืองนราธิวาส,960101,บางนาค,1,ชั้นล่างอาคารเรียน 2 โรงเรียนเทศบาล 5,NaN,NaN,เมืองนราธิวาส
81425,96,นราธิวาส,9699,ท้องถิ่นเทศบาลเมืองนราธิวาส,960101,บางนาค,1,ศาลาวัดประชาภิรมย์ 1,NaN,NaN,เมืองนราธิวาส


,ADM3_EN,ADM3_TH,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE
0,Phraborom Maharatchawang,พระบรมมหาราชวัง,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
1,Wang Burapha Phirom,วังบูรพาภิรมย์,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
2,Wat Ratchabophit,วัดราชบพิธ,TH100103,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
3,Samran Rat,สำราญราษฎร์,TH100104,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
4,San Chaopho Suea,ศาลเจ้าพ่อเสือ,TH100105,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH


In [237]:
# Join df by df2 on subdistrict proposed_district and province
# !! Some subdistrict and province has multiple matches, causing duplicated rows !!
df_merged = pd.merge(df, df2, left_on=['subdistrict', 'proposed_district', 'province'], right_on=['ADM3_TH', 'ADM2_TH', 'ADM1_TH'], how='left')

# If missing ADM3_EN, join by subdistrict and province on the missing rows
df_merged_v2 = pd.merge(df, df2, left_on=['subdistrict', 'province'], right_on=['ADM3_TH', 'ADM1_TH'], how='left')
df_merged.update(df_merged_v2)
display(df_merged)
display(df_merged[df_merged['ADM3_EN'].isna()])

,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,...,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE
0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช,13.752802,100.489581,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
1,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์,13.755687,100.490966,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
2,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,13.745237,100.495748,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
3,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดพระเชตุพน ถนนมหาราช,13.745582,100.491293,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
4,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100102,วังบูรพาภิรมย์,1,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,13.743462,100.498851,...,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81422,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านลาหงา,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH
81423,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนบ้านบากันโต๊ะทิด,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH
81424,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนบ้านนาพญา,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH
81425,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านคลองขุด,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH


,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,...,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE


In [214]:
# Try one row
n = 12768
row = df_merged.iloc[n]
if row['province'] == "กรุงเทพมหานคร":
    address = f"{row['location']} แขวง{row['subdistrict']} เขต{row['ADM2_TH']} {row['province']} ประเทศไทย"
else:
    address = f"{row['location']} ตำบล{row['subdistrict']} อำเภอ{row['ADM2_TH']} จังหวัด{row['province']} ประเทศไทย"
print(address)

# Geocode function
def address_to_id(address, api_key=APIKEY):
    base_url = "https://places.googleapis.com/v1/places:searchText"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.id,places.name,nextPageToken",
    }
    body = {
        "textQuery": address,
        "languageCode": "th",
    }
    
    response = requests.post(base_url, headers=headers, json=body)
    if response.status_code == 200:
        result = response.json()
        if 'places' in result and len(result['places']) > 0:
            place_id = result['places'][0]['id']
            return place_id, result
        else:
            return None
    else:
        logging.error(f"Error {response.status_code}: {response.text}")
        return None
    
# Test the function
try:
    place_id, result = address_to_id(address)
except Exception as e:
    print(f"Error: {e}")
    place_id, result = None, None

ศาลาอเนกประสงค์บ้านบางอิน ตำบลคลองใหญ่ อำเภอแหลมงอบ จังหวัดตราด ประเทศไทย


In [215]:
display(place_id, result)
place_url = f"https://www.google.com/maps/place/?q=place_id:{place_id}"
print(place_url)

'ChIJp9Ov0-JDBDER7WRRYuCMYHY'

{'places': [{'name': 'places/ChIJp9Ov0-JDBDER7WRRYuCMYHY',
   'id': 'ChIJp9Ov0-JDBDER7WRRYuCMYHY'},
  {'name': 'places/ChIJRY8W6dhqBDERWRH8SBiaPSA',
   'id': 'ChIJRY8W6dhqBDERWRH8SBiaPSA'}]}

https://www.google.com/maps/place/?q=place_id:ChIJp9Ov0-JDBDER7WRRYuCMYHY


In [238]:
# Run through all rows and save placeid_results
# Divide to 2 chunks
chunk_size = len(df_merged) // 2
testchunk = df_merged.iloc[0:5]
chunkn1 = df_merged.iloc[:chunk_size]
chunkn2 = df_merged.iloc[chunk_size:]
display(testchunk, chunkn1, chunkn2)

,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,...,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE
0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช,13.752802,100.489581,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
1,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์,13.755687,100.490966,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
2,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,13.745237,100.495748,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
3,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดพระเชตุพน ถนนมหาราช,13.745582,100.491293,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
4,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100102,วังบูรพาภิรมย์,1,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,13.743462,100.498851,...,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH


,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,...,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE
0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช,13.752802,100.489581,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
1,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์,13.755687,100.490966,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
2,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,13.745237,100.495748,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
3,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดพระเชตุพน ถนนมหาราช,13.745582,100.491293,...,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
4,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100102,วังบูรพาภิรมย์,1,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,13.743462,100.498851,...,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40708,41,อุดรธานี,4197,ท้องถิ่นเทศบาลตำบลนิคมสงเคราะห์,410119,โคกสะอาด,9,ศาลากลางบ้านนิคมฯ 2 หมู่ที่ 6 (หลังใหม่) ต.โคก...,NaN,NaN,...,TH410119,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40709,41,อุดรธานี,4197,ท้องถิ่นเทศบาลตำบลนิคมสงเคราะห์,410119,โคกสะอาด,9,ศาลากลางบ้านนิคมฯ 3 (ที่่ทำการ อบต.โคกสะอาด หล...,NaN,NaN,...,TH410119,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40710,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,ศาลาวัดนาคาเทวี (นาข่า),NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40711,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,ศาลากลางบ้านนาข่า,NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH


,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,...,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE
40713,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,ศาลาวัดโคกศรีสำราญ (ถ่อนใหญ่),NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40714,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,ศาลากลางบ้านถ่อนใหญ่,NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40715,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,สวนสาธารณะหนองภาโรน (ดอนแตง),NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40716,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,ศาลาวัดอัมพวัน (ดงยวด),NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
40717,41,อุดรธานี,4198,ท้องถิ่นเทศบาลตำบลนาข่า,410116,นาข่า,2,ศาลากลางบ้านดงยวด,NaN,NaN,...,TH410116,Mueang Udon Thani,เมืองอุดรธานี,TH4101,Udon Thani,อุดรธานี,TH41,Thailand,ประเทศไทย,TH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81422,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านลาหงา,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH
81423,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนบ้านบากันโต๊ะทิด,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH
81424,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนบ้านนาพญา,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH
81425,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านคลองขุด,NaN,NaN,...,TH910502,La-Ngu,ละงู,TH9105,Satun,สตูล,TH91,Thailand,ประเทศไทย,TH


In [ ]:
# Run through all rows in chunk
chunk = chunkn2
chunk_name = "chunk_2"
placeid_results = []
rate_limit_per_minute = 600  # Adjust as needed
from tqdm import tqdm
for index, row in tqdm(chunk.iterrows(), total=chunk.shape[0]):
    if row['province'] == "กรุงเทพมหานคร":
        address = f"{row['location']} แขวง{row['subdistrict']} เขต{row['ADM2_TH']} {row['province']} ประเทศไทย"
    else:
        address = f"{row['location']} ตำบล{row['subdistrict']} อำเภอ{row['ADM2_TH']} จังหวัด{row['province']} ประเทศไทย"
    try:
        place_id, result = address_to_id(address)
        placeid_results.append({
            'index': index,
            'location': row['location'],
            'address': address,
            'place_id': place_id,
            'place_url': f"https://www.google.com/maps/place/?q=place_id:{place_id}" if place_id else None,
            'result': result
        })
    except Exception as e:
        # logging.error(f"Error processing row {index} with address '{address}': {e}")
        placeid_results.append({
            'index': index,
            'location': row['location'],
            'address': address,
            'place_id': None,
            'place_url': None,
            'result': str(e)
        })
    # Sleep to respect rate limit
    # time.sleep(60.0 / rate_limit_per_minute)  # Adjust sleep time as needed to respect API rate limits
    
df_placeid = pd.DataFrame(placeid_results)
display(df_placeid)

  0%|          | 0/40714 [00:00<?, ?it/s]

  1%|          | 442/40714 [03:58<5:14:45,  2.13it/s]ERROR:root:Error 503: {
  "error": {
    "code": 503,
    "message": "The service is currently unavailable.",
    "status": "UNAVAILABLE"
  }
}

 89%|████████▉ | 36278/40714 [5:07:38<42:56,  1.72it/s]  ERROR:root:Error 500: {
  "error": {
    "code": 500,
    "message": "Internal server error. Please retry. If the problem persists, please open a support case. https://goo.gle/maps-platform-support",
    "status": "INTERNAL"
  }
}

 98%|█████████▊| 39899/40714 [5:39:38<08:03,  1.69it/s]ERROR:root:Error 503: {
  "error": {
    "code": 503,
    "message": "The service is currently unavailable.",
    "status": "UNAVAILABLE"
  }
}

100%|██████████| 40714/40714 [5:47:31<00:00,  1.95it/s]


,index,location,address,place_id,place_url,result
0,40713,ศาลาวัดโคกศรีสำราญ (ถ่อนใหญ่),ศาลาวัดโคกศรีสำราญ (ถ่อนใหญ่) ตำบลนาข่า อำเภอเ...,ChIJ1ypwINwDIzER94opaptA7CU,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJ1ypwINwDIzER9...
1,40714,ศาลากลางบ้านถ่อนใหญ่,ศาลากลางบ้านถ่อนใหญ่ ตำบลนาข่า อำเภอเมืองอุดรธ...,ChIJgzq962udIzER3rT6PSY3Pj4,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJgzq962udIzER3...
2,40715,สวนสาธารณะหนองภาโรน (ดอนแตง),สวนสาธารณะหนองภาโรน (ดอนแตง) ตำบลนาข่า อำเภอเม...,ChIJH-oOjdZ3IzERQh4izHqyIQA,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJH-oOjdZ3IzERQ...
3,40716,ศาลาวัดอัมพวัน (ดงยวด),ศาลาวัดอัมพวัน (ดงยวด) ตำบลนาข่า อำเภอเมืองอุด...,ChIJ9f2qV7F4IzERRkrpVHZ-ikI,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJ9f2qV7F4IzERR...
4,40717,ศาลากลางบ้านดงยวด,ศาลากลางบ้านดงยวด ตำบลนาข่า อำเภอเมืองอุดรธานี...,ChIJqSPgy3UzIzERASyi5uUujYQ,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJqSPgy3UzIzERA...
...,...,...,...,...,...,...
40709,81422,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านลาหงา,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านลาหงา ตำบล...,ChIJUwsgAx1RTDARdJZa2A8DiUw,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJUwsgAx1RTDARd...
40710,81423,อาคารเรียนโรงเรียนบ้านบากันโต๊ะทิด,อาคารเรียนโรงเรียนบ้านบากันโต๊ะทิด ตำบลละงู อำ...,ChIJjTZbK5xZTDARvUcSdEjIA4U,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJjTZbK5xZTDARv...
40711,81424,อาคารเรียนโรงเรียนบ้านนาพญา,อาคารเรียนโรงเรียนบ้านนาพญา ตำบลละงู อำเภอละงู...,ChIJyT9LXD5XTDAR77MI9TahETE,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJyT9LXD5XTDAR7...
40712,81425,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านคลองขุด,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านคลองขุด ตำ...,ChIJl1EcgspWTDARegq72vCA-qc,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJl1EcgspWTDARe...


In [240]:
# Count successful and failed requests
success_count = df_placeid['place_id'].notna().sum()
failure_count = df_placeid['place_id'].isna().sum()
all_count = len(df_placeid)
print(f"Successful requests: {success_count} ({success_count/all_count:.2%})")
print(f"Failed requests: {failure_count} ({failure_count/all_count:.2%})")

Successful requests: 37339 (91.71%)
Failed requests: 3375 (8.29%)


In [241]:
df_placeid

,index,location,address,place_id,place_url,result
0,40713,ศาลาวัดโคกศรีสำราญ (ถ่อนใหญ่),ศาลาวัดโคกศรีสำราญ (ถ่อนใหญ่) ตำบลนาข่า อำเภอเ...,ChIJ1ypwINwDIzER94opaptA7CU,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJ1ypwINwDIzER9...
1,40714,ศาลากลางบ้านถ่อนใหญ่,ศาลากลางบ้านถ่อนใหญ่ ตำบลนาข่า อำเภอเมืองอุดรธ...,ChIJgzq962udIzER3rT6PSY3Pj4,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJgzq962udIzER3...
2,40715,สวนสาธารณะหนองภาโรน (ดอนแตง),สวนสาธารณะหนองภาโรน (ดอนแตง) ตำบลนาข่า อำเภอเม...,ChIJH-oOjdZ3IzERQh4izHqyIQA,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJH-oOjdZ3IzERQ...
3,40716,ศาลาวัดอัมพวัน (ดงยวด),ศาลาวัดอัมพวัน (ดงยวด) ตำบลนาข่า อำเภอเมืองอุด...,ChIJ9f2qV7F4IzERRkrpVHZ-ikI,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJ9f2qV7F4IzERR...
4,40717,ศาลากลางบ้านดงยวด,ศาลากลางบ้านดงยวด ตำบลนาข่า อำเภอเมืองอุดรธานี...,ChIJqSPgy3UzIzERASyi5uUujYQ,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJqSPgy3UzIzERA...
...,...,...,...,...,...,...
40709,81422,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านลาหงา,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านลาหงา ตำบล...,ChIJUwsgAx1RTDARdJZa2A8DiUw,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJUwsgAx1RTDARd...
40710,81423,อาคารเรียนโรงเรียนบ้านบากันโต๊ะทิด,อาคารเรียนโรงเรียนบ้านบากันโต๊ะทิด ตำบลละงู อำ...,ChIJjTZbK5xZTDARvUcSdEjIA4U,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJjTZbK5xZTDARv...
40711,81424,อาคารเรียนโรงเรียนบ้านนาพญา,อาคารเรียนโรงเรียนบ้านนาพญา ตำบลละงู อำเภอละงู...,ChIJyT9LXD5XTDAR77MI9TahETE,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJyT9LXD5XTDAR7...
40712,81425,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านคลองขุด,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านคลองขุด ตำ...,ChIJl1EcgspWTDARegq72vCA-qc,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJl1EcgspWTDARe...


In [255]:
# Get latitude and longitude from placeID using geocode API
def id_to_position(place_id, api_key=APIKEY):
    base_url = f"https://geocode.googleapis.com/v4beta/geocode/places/{place_id}?key={api_key}"
    response = requests.get(base_url)
    if response.status_code != 200:
        logging.error(f"Error {response.status_code}: {response.text}")
        return None
    else:
        return response.json()

In [245]:
# Load placeid all chunks
from glob import glob
placeid_name = "station66_placeid_chunk_*.csv"
df_placeid = pd.concat([pd.read_csv(f) for f in sorted(glob(placeid_name))], ignore_index=True)
df_placeid

,index,location,address,place_id,place_url,result
0,0,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช แขวงพระบ...,ChIJsQW2s6CZ4jARKCwV_40G50c,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJsQW2s6CZ4jARK...
1,1,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์ แขวงพระบรมมหาร...,ChIJc3ySHQWZ4jAR8ocYUwGc9mM,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJc3ySHQWZ4jAR8...
2,2,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,ChIJV_-yxQSZ4jAR2GWdpHhQYVM,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJV_-yxQSZ4jAR2...
3,3,โรงเรียนวัดพระเชตุพน ถนนมหาราช,โรงเรียนวัดพระเชตุพน ถนนมหาราช แขวงพระบรมมหารา...,ChIJCRWKpOmZ4jARCM4KBhoWp6A,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJCRWKpOmZ4jARC...
4,4,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,ChIJaRiqxauZ4jARilfgXfsAobU,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJaRiqxauZ4jARi...
...,...,...,...,...,...,...
81422,81422,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านลาหงา,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านลาหงา ตำบล...,ChIJUwsgAx1RTDARdJZa2A8DiUw,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJUwsgAx1RTDARd...
81423,81423,อาคารเรียนโรงเรียนบ้านบากันโต๊ะทิด,อาคารเรียนโรงเรียนบ้านบากันโต๊ะทิด ตำบลละงู อำ...,ChIJjTZbK5xZTDARvUcSdEjIA4U,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJjTZbK5xZTDARv...
81424,81424,อาคารเรียนโรงเรียนบ้านนาพญา,อาคารเรียนโรงเรียนบ้านนาพญา ตำบลละงู อำเภอละงู...,ChIJyT9LXD5XTDAR77MI9TahETE,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJyT9LXD5XTDAR7...
81425,81425,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านคลองขุด,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านคลองขุด ตำ...,ChIJl1EcgspWTDARegq72vCA-qc,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJl1EcgspWTDARe...


In [249]:
df_with_placeid = df_merged.join(df_placeid.set_index('index'), how='left', rsuffix='_placeid')
df_with_placeid

,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,...,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE,location_placeid,address,place_id,place_url,result
0,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช,13.752802,100.489581,...,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช,หอประชุม มหาวิทยาลัยศิลปากร ถนนมหาราช แขวงพระบ...,ChIJsQW2s6CZ4jARKCwV_40G50c,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJsQW2s6CZ4jARK...
1,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์,13.755687,100.490966,...,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์,โรงเรียนวัดมหาธาตุ ถนนพระจันทร์ แขวงพระบรมมหาร...,ChIJc3ySHQWZ4jAR8ocYUwGc9mM,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJc3ySHQWZ4jAR8...
2,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,13.745237,100.495748,...,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,ศาลาหน้าอาคารวาสนะประทานวิทยาสิทธิ์ 1 โรงเรียน...,ChIJV_-yxQSZ4jAR2GWdpHhQYVM,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJV_-yxQSZ4jAR2...
3,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100101,พระบรมมหาราชวัง,1,โรงเรียนวัดพระเชตุพน ถนนมหาราช,13.745582,100.491293,...,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,โรงเรียนวัดพระเชตุพน ถนนมหาราช,โรงเรียนวัดพระเชตุพน ถนนมหาราช แขวงพระบรมมหารา...,ChIJCRWKpOmZ4jARCM4KBhoWp6A,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJCRWKpOmZ4jARC...
4,10,กรุงเทพมหานคร,1001,ท้องถิ่นเขตพระนคร,100102,วังบูรพาภิรมย์,1,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,13.743462,100.498851,...,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,ตึกสามัคยาจารย์ โรงเรียนสวนกุหลาบวิทยาลัย ถนนต...,ChIJaRiqxauZ4jARilfgXfsAobU,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJaRiqxauZ4jARi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81422,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านลาหงา,NaN,NaN,...,สตูล,TH91,Thailand,ประเทศไทย,TH,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านลาหงา,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านลาหงา ตำบล...,ChIJUwsgAx1RTDARdJZa2A8DiUw,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJUwsgAx1RTDARd...
81423,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนบ้านบากันโต๊ะทิด,NaN,NaN,...,สตูล,TH91,Thailand,ประเทศไทย,TH,อาคารเรียนโรงเรียนบ้านบากันโต๊ะทิด,อาคารเรียนโรงเรียนบ้านบากันโต๊ะทิด ตำบลละงู อำ...,ChIJjTZbK5xZTDARvUcSdEjIA4U,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJjTZbK5xZTDARv...
81424,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนบ้านนาพญา,NaN,NaN,...,สตูล,TH91,Thailand,ประเทศไทย,TH,อาคารเรียนโรงเรียนบ้านนาพญา,อาคารเรียนโรงเรียนบ้านนาพญา ตำบลละงู อำเภอละงู...,ChIJyT9LXD5XTDAR77MI9TahETE,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJyT9LXD5XTDAR7...
81425,91,สตูล,9105,อำเภอละงู,910502,ละงู,2,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านคลองขุด,NaN,NaN,...,สตูล,TH91,Thailand,ประเทศไทย,TH,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านคลองขุด,อาคารเรียนโรงเรียนสอนศาสนามัสยิดบ้านคลองขุด ตำ...,ChIJl1EcgspWTDARegq72vCA-qc,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJl1EcgspWTDARe...


In [258]:
# Contribute search for place_ids in a selected province
sel_province = "นนทบุรี"
sel_place_ids = df_placeid[df_merged["province"] == sel_province]["place_id"].unique()
# Select only place_ids start with "ChIJ"
sel_place_ids = [pid for pid in sel_place_ids if str(pid).startswith("ChIJ")]
display(sel_place_ids, len(sel_place_ids))

['ChIJi13l_QKb4jARoHtRLn5zUU4',
 'ChIJMYXXB2qb4jARr0eTzasz0c4',
 'ChIJcbIeJ7ub4jARLcMxvr90Pow',
 'ChIJs0PnRKqb4jARMXBS1HEo9zM',
 'ChIJmVA3TKqb4jARN17OjfT926g',
 'ChIJVevJsqeb4jARvQ7r-5xc5LE',
 'ChIJwcFivKeb4jARP_cigXAfqJE',
 'ChIJOyTBXKab4jARrInT9E_WkgU',
 'ChIJ_1Eka6Sb4jARMRvYRDpFr2I',
 'ChIJ5ebG1bGb4jARlF7EMUWoZcA',
 'ChIJ_b_bJACb4jARDJkLc_5Vmoc',
 'ChIJEU7de4mb4jARFRDdfhNaVV8',
 'ChIJvU2cm4ya4jARsQPfOq4mdXI',
 'ChIJaxm76I-a4jARI8iT8mz3dlk',
 'ChIJ04sD-3eQ4jARsLlv_olwmT0',
 'ChIJk1UIoU6F4jAR4mdQndNMwpI',
 'ChIJb62oR_ia4jARbVGnsDVl5Lg',
 'ChIJVVVVVZKa4jARezWND7OQYZ8',
 'ChIJm69tBZaa4jAR3YSTYilgRa0',
 'ChIJRxy29Q6b4jARyezTHPRLgK8',
 'ChIJp0JNoCSb4jARbpWN899iJbI',
 'ChIJ-5KEVVeb4jARL9s6HrJcaAQ',
 'ChIJhQRXJp-a4jARJAAz5g8Y090',
 'ChIJd2-ZPq2a4jARSAEfGZGZUsM',
 'ChIJ5b1Ur7ya4jARimKcQ6egzvI',
 'ChIJ7SXNgLqa4jARHlFj97HElKE',
 'ChIJVRf77bWa4jARdwne73TMhwo',
 'ChIJh7iVqk-b4jARZGmIUk8w8m4',
 'ChIJ0xBXK7Sa4jARJGg_GhaLyAA',
 'ChIJiZSNQk-b4jARn1DCRe1v560',
 'ChIJQ-149UuF4jARTr_9aN29XpM',
 'ChIJY8

672

In [ ]:
# Run through selected place_ids by province
geodata = []
for pid in tqdm(X, total=len(X)):
    try:
        result = id_to_position(pid)
        geodata.append({
            'place_id': pid,
            'geocode_result': result,
        })
    except Exception as e:
        geodata.append({
            'place_id': pid,
            'geocode_result': str(e),
        })
df_geodata = pd.DataFrame(geodata)

100%|██████████| 215/215 [05:22<00:00,  1.50s/it]


In [308]:
# Load geocode of province
df_geocode = pd.read_csv(f'station66_geocode_{sel_province}_v1.csv')[['place_id', 'proposed_latitude', 'proposed_longitude', 'geocode_result']]
df_geocode = df_geocode[df_geocode['proposed_latitude'].notna()]
# Append with df_geocode_v2
df_geocode_v2 = pd.read_csv(f'station66_geocode_{sel_province}_v2.csv')[['place_id', 'geocode_result']]
df_geocode = pd.concat([df_geocode, df_geocode_v2], ignore_index=True)[['place_id', 'geocode_result']]
df_geocode

,place_id,geocode_result
0,ChIJi13l_QKb4jARoHtRLn5zUU4,{'place': '//places.googleapis.com/places/ChIJ...
1,ChIJMYXXB2qb4jARr0eTzasz0c4,{'place': '//places.googleapis.com/places/ChIJ...
2,ChIJs0PnRKqb4jARMXBS1HEo9zM,{'place': '//places.googleapis.com/places/ChIJ...
3,ChIJmVA3TKqb4jARN17OjfT926g,{'place': '//places.googleapis.com/places/ChIJ...
4,ChIJVevJsqeb4jARvQ7r-5xc5LE,{'place': '//places.googleapis.com/places/ChIJ...
...,...,...
667,ChIJH7026kuD4jARwhx1zqnTogg,{'place': '//places.googleapis.com/places/ChIJ...
668,ChIJiSV8OrOD4jAR6Fy9KDB5kWQ,{'place': '//places.googleapis.com/places/ChIJ...
669,ChIJBVRv0wqZ4jARFmpjWvbKKy8,{'place': '//places.googleapis.com/places/ChIJ...
670,ChIJk0s37FWD4jARdgGbiolSMwM,{'place': '//places.googleapis.com/places/ChIJ...


In [409]:
# Create columns
import ast
df_geocode_super = []
for i in tqdm(range(len(df_geocode))):
    row = df_geocode.iloc[i]
    geocode_result = ast.literal_eval(row['geocode_result']) if isinstance(row['geocode_result'], str) else row['geocode_result']
    try: this_lat = geocode_result['location']['latitude'] 
    except: this_lat = None
    try: this_long = geocode_result['location']['longitude'] 
    except: this_long = None
    df_geocode_super.append({
        'place_id': row['place_id'],
        'proposed_latitude': this_lat,
        'proposed_longitude': this_long,
    })
df_geocode = pd.DataFrame(df_geocode_super)
df_geocode

100%|██████████| 672/672 [00:00<00:00, 43635.88it/s]


,place_id,proposed_latitude,proposed_longitude
0,ChIJi13l_QKb4jARoHtRLn5zUU4,13.836347,100.487320
1,ChIJMYXXB2qb4jARr0eTzasz0c4,13.825872,100.487249
2,ChIJs0PnRKqb4jARMXBS1HEo9zM,13.825948,100.487977
3,ChIJmVA3TKqb4jARN17OjfT926g,13.825326,100.488274
4,ChIJVevJsqeb4jARvQ7r-5xc5LE,13.824239,100.497747
...,...,...,...
667,ChIJH7026kuD4jARwhx1zqnTogg,13.875873,100.547884
668,ChIJiSV8OrOD4jAR6Fy9KDB5kWQ,13.877165,100.531160
669,ChIJBVRv0wqZ4jARFmpjWvbKKy8,13.759443,100.487063
670,ChIJk0s37FWD4jARdgGbiolSMwM,13.876499,100.530543


In [412]:
# Clean df_geocode
# Adjust precision to 6 decimal places
df_geocode["proposed_latitude"] = pd.to_numeric(df_geocode["proposed_latitude"], errors='coerce').round(7)
df_geocode["proposed_longitude"] = pd.to_numeric(df_geocode["proposed_longitude"], errors='coerce').round(7)
df_geocode

# SAVE to CSV
df_geocode.to_csv(f'station66_geocode_{sel_province}_cleaned.csv', index=False)

In [413]:
# Load cleaned df_geocode
df_geocode = pd.read_csv(f'station66_geocode_{sel_province}_cleaned.csv')

# Join df_geocode with df_merged on place_id
df_merged_with_placeid = df_merged.join(df_placeid.set_index('index'), how='left', rsuffix='_placeid')[df_merged["province"] == sel_province]
df_merged_with_geocode = df_merged_with_placeid.merge(df_geocode, on='place_id', how='left')
df_merged_with_geocode

,provinceNumber,province,registrar_code,registrar,subdis_code,subdistrict,electorate,location,latitude,longitude,...,ADM0_EN,ADM0_TH,ADM0_PCODE,location_placeid,address,place_id,place_url,result,proposed_latitude,proposed_longitude
0,12,นนทบุรี,1201,อำเภอเมืองนนทบุรี,120106,บางไผ่,3,ศาลาการเปรียญวัดอมฤต,13.836347,100.487320,...,Thailand,ประเทศไทย,TH,ศาลาการเปรียญวัดอมฤต,ศาลาการเปรียญวัดอมฤต ตำบลบางไผ่ อำเภอเมืองนนทบ...,ChIJi13l_QKb4jARoHtRLn5zUU4,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJi13l_QKb4jARo...,13.836347,100.487320
1,12,นนทบุรี,1201,อำเภอเมืองนนทบุรี,120106,บางไผ่,3,ลานเอนกประสงค์ปากทางเข้า อบต.บางไผ่,13.824888,100.493056,...,Thailand,ประเทศไทย,TH,ปะรำลานเอนกประสงค์ปากทางเข้า อบต.บางไผ่,ปะรำลานเอนกประสงค์ปากทางเข้า อบต.บางไผ่ ตำบลบา...,ChIJMYXXB2qb4jARr0eTzasz0c4,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJMYXXB2qb4jARr...,13.825872,100.487249
2,12,นนทบุรี,1201,อำเภอเมืองนนทบุรี,120106,บางไผ่,3,ลานเอนกประสงค์ปากทางเข้า อบต.บางไผ่,13.830225,100.485326,...,Thailand,ประเทศไทย,TH,ปะรำลานเอนกประสงค์ปากทางเข้า อบต.บางไผ่,ปะรำลานเอนกประสงค์ปากทางเข้า อบต.บางไผ่ ตำบลบา...,ChIJMYXXB2qb4jARr0eTzasz0c4,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJMYXXB2qb4jARr...,13.825872,100.487249
3,12,นนทบุรี,1201,อำเภอเมืองนนทบุรี,120106,บางไผ่,3,ลานเอนกประสงค์ใต้สะพานพระราม 5,13.835265,100.502688,...,Thailand,ประเทศไทย,TH,ปะรำลานเอนกประสงค์ใต้สะพานพระราม 5,ปะรำลานเอนกประสงค์ใต้สะพานพระราม 5 ตำบลบางไผ่ ...,ChIJcbIeJ7ub4jARLcMxvr90Pow,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJcbIeJ7ub4jARL...,13.832619,100.495534
4,12,นนทบุรี,1201,อำเภอเมืองนนทบุรี,120106,บางไผ่,3,ลานเอนกประสงค์ใต้สะพานพระราม 5,13.830879,100.491807,...,Thailand,ประเทศไทย,TH,ปะรำลานเอนกประสงค์ใต้สะพานพระราม 5,ปะรำลานเอนกประสงค์ใต้สะพานพระราม 5 ตำบลบางไผ่ ...,ChIJcbIeJ7ub4jARLcMxvr90Pow,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJcbIeJ7ub4jARL...,13.832619,100.495534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,12,นนทบุรี,1299,ท้องถิ่นเทศบาลนครนนทบุรี,120105,ท่าทราย,1,ที่ว่างซอยประชานิเวศน์ 3 ซอย 8/3 แยก 12 (สนามต...,13.877911,100.527045,...,Thailand,ประเทศไทย,TH,ปะรำบริเวณที่ว่างซอยประชานิเวศน์ 3 ซอย 8/3 แยก...,ปะรำบริเวณที่ว่างซอยประชานิเวศน์ 3 ซอย 8/3 แยก...,NaN,NaN,cannot unpack non-iterable NoneType object,NaN,NaN
896,12,นนทบุรี,1299,ท้องถิ่นเทศบาลนครนนทบุรี,120105,ท่าทราย,1,ทางเท้าหน้าปากซอยประชานิเวศน์ 3 ซอย 8/8,13.879341,100.524910,...,Thailand,ประเทศไทย,TH,ปะรำบริเวณทางเท้าหน้าปากซอยประชานิเวศน์ 3 ซอย 8/8,ปะรำบริเวณทางเท้าหน้าปากซอยประชานิเวศน์ 3 ซอย ...,ChIJiSV8OrOD4jAR6Fy9KDB5kWQ,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJiSV8OrOD4jAR6...,13.877165,100.531160
897,12,นนทบุรี,1299,ท้องถิ่นเทศบาลนครนนทบุรี,120105,ท่าทราย,1,ลานออกกำลังกายซอยประชานิเวศน์ 3 ซอย 12 (หลังธน...,13.872465,100.530835,...,Thailand,ประเทศไทย,TH,ปะรำบริเวณลานออกกำลังกายซอยประชานิเวศน์ 3 ซอย ...,ปะรำบริเวณลานออกกำลังกายซอยประชานิเวศน์ 3 ซอย ...,ChIJk0s37FWD4jARdgGbiolSMwM,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJk0s37FWD4jARd...,13.876499,100.530543
898,12,นนทบุรี,1299,ท้องถิ่นเทศบาลนครนนทบุรี,120105,ท่าทราย,1,ห้องโถงอาคารสำนักงานแขวงท่าทราย,13.854917,100.544459,...,Thailand,ประเทศไทย,TH,บริเวณห้องโถงอาคารสำนักงานแขวงท่าทราย,บริเวณห้องโถงอาคารสำนักงานแขวงท่าทราย ตำบลท่าท...,ChIJaas4J6uc4jARiTkktwOG06g,https://www.google.com/maps/place/?q=place_id:...,{'places': [{'name': 'places/ChIJaas4J6uc4jARi...,13.870945,100.532683


In [414]:
# Calculate distance from Tambon epicenter to proposed lat/long using Haversine formula
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # Haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Radius of earth in kilometers. Use 3956 for miles
    return c * r